In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
error_lines_bad = False
data = pd.read_json("data_5scheduler.json")
data

,title,identifier,description,source,credits,instructors,offered,prerequisites,corequisites,currently_offered,fee
0,Introduction to American Cultures,AMST-103-HM,An interdisciplinary introduction to principal...,HarveyMudd,300,[Staff],,,,False,0
1,Print and American Culture,AMST-115-HM,Covers numerous developments in American print...,HarveyMudd,300,[Anup Gampa],,,,True,0
2,Hyphenated Americans,AMST-120-HM,A focus on the experience of immigrants in the...,HarveyMudd,300,[Balseiro],,,,False,0
3,"Life: Knowledge, Belief, and Cultural Practices",ANTH-110-HM,An exploration of cultural attitudes toward li...,HarveyMudd,300,[de Laet],,,,False,0
4,Introduction to the Anthropology of Science an...,ANTH-111-HM,An introduction to science and technology as c...,HarveyMudd,300,[Marianne De Laet],,,,True,0
...,...,...,...,...,...,...,...,...,...,...,...
4438,Autobiography and Memoir,WRIT-126-PZ,This course will look at the writer’s life as ...,Pitzer,0,[],,,,False,0
4439,Writing the Senior Thesis,WRIT-195-PZ,This course will guide you through the process...,Pitzer,0,[],,,,False,0
4440,IGLAS Seminar,GLAS-185-PZ,,Pitzer,0,[],,,,False,0
4441,Global Local Research Workshop,GLAS-19-4A,,Pitzer,0,[],,,,False,0


In [33]:
data['index'] = range(1, len(data) + 1)
data

,title,identifier,description,source,credits,instructors,offered,prerequisites,corequisites,currently_offered,fee,index_col,index
0,Introduction to American Cultures,AMST-103-HM,An interdisciplinary introduction to principal...,HarveyMudd,300,[Staff],,,,False,0,0,1
1,Print and American Culture,AMST-115-HM,Covers numerous developments in American print...,HarveyMudd,300,[Anup Gampa],,,,True,0,1,2
2,Hyphenated Americans,AMST-120-HM,A focus on the experience of immigrants in the...,HarveyMudd,300,[Balseiro],,,,False,0,2,3
3,"Life: Knowledge, Belief, and Cultural Practices",ANTH-110-HM,An exploration of cultural attitudes toward li...,HarveyMudd,300,[de Laet],,,,False,0,3,4
4,Introduction to the Anthropology of Science an...,ANTH-111-HM,An introduction to science and technology as c...,HarveyMudd,300,[Marianne De Laet],,,,True,0,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4438,Autobiography and Memoir,WRIT-126-PZ,This course will look at the writer’s life as ...,Pitzer,0,[],,,,False,0,4438,4439
4439,Writing the Senior Thesis,WRIT-195-PZ,This course will guide you through the process...,Pitzer,0,[],,,,False,0,4439,4440
4440,IGLAS Seminar,GLAS-185-PZ,,Pitzer,0,[],,,,False,0,4440,4441
4441,Global Local Research Workshop,GLAS-19-4A,,Pitzer,0,[],,,,False,0,4441,4442


In [51]:
# Checking shape of the data
data.shape

(4443, 13)

In [35]:
# Checking dataset
data.head(2)

,title,identifier,description,source,credits,instructors,offered,prerequisites,corequisites,currently_offered,fee,index_col,index
0,Introduction to American Cultures,AMST-103-HM,An interdisciplinary introduction to principal...,HarveyMudd,300,[Staff],,,,False,0,0,1
1,Print and American Culture,AMST-115-HM,Covers numerous developments in American print...,HarveyMudd,300,[Anup Gampa],,,,True,0,1,2


In [36]:
# Checking list of variables in the data
data.columns

Index(['title', 'identifier', 'description', 'source', 'credits',
       'instructors', 'offered', 'prerequisites', 'corequisites',
       'currently_offered', 'fee', 'index_col', 'index'],
      dtype='object')

In [37]:
# Creating data with important features
imp_features = ['title', 'identifier', 'description', 'source']
data1 = data[imp_features]
data1.head()

,title,identifier,description,source
0,Introduction to American Cultures,AMST-103-HM,An interdisciplinary introduction to principal...,HarveyMudd
1,Print and American Culture,AMST-115-HM,Covers numerous developments in American print...,HarveyMudd
2,Hyphenated Americans,AMST-120-HM,A focus on the experience of immigrants in the...,HarveyMudd
3,"Life: Knowledge, Belief, and Cultural Practices",ANTH-110-HM,An exploration of cultural attitudes toward li...,HarveyMudd
4,Introduction to the Anthropology of Science an...,ANTH-111-HM,An introduction to science and technology as c...,HarveyMudd


In [38]:
# Checking missing values
data1.isnull().sum()

title          0
identifier     0
description    0
source         0
dtype: int64

In [39]:
# Replacing missing values
# not sure if this is needed
missing_var = data1.isnull().sum().keys()
for i in missing_var:
    data1[i] = data1[i].fillna('')

/var/folders/n4/wr2pjk454g5cxqdbd2qywndc0000gn/T/ipykernel_39126/2770977688.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1[i] = data1[i].fillna('')


In [40]:
# Checking missing values
data1.isnull().sum()

title          0
identifier     0
description    0
source         0
dtype: int64

In [41]:
# Combining the values of the all features
data1['Combined_features'] = data1['title']+' '+data1['identifier']+' '+data1['description']+' '+data1['source']

/var/folders/n4/wr2pjk454g5cxqdbd2qywndc0000gn/T/ipykernel_39126/1213261718.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['Combined_features'] = data1['title']+' '+data1['identifier']+' '+data1['description']+' '+data1['source']


In [42]:
#Checking first five values of the combined features
data1['Combined_features'][0:5]

0    Introduction to American Cultures AMST-103-HM ...
1    Print and American Culture AMST-115-HM Covers ...
2    Hyphenated Americans AMST-120-HM A focus on th...
3    Life: Knowledge, Belief, and Cultural Practice...
4    Introduction to the Anthropology of Science an...
Name: Combined_features, dtype: object

In [43]:
#Creating an object of count victorizer
cv = CountVectorizer()

In [44]:
#Creating count vectorization of combined features
count_matrix = cv.fit_transform(data1['Combined_features'])

In [45]:
#Computing cosine similarity of count matrix
cosin_sim = cosine_similarity(count_matrix)

In [46]:
#Checking first five values cosine similarity
cosin_sim[5]

array([0.0862796 , 0.25951446, 0.5668485 , ..., 0.        , 0.        ,
       0.        ])

In [49]:
#Defining function for course recommendation system
def get_recommender(course_title):
    
    #Finding an index number of given movie title which you like most
    course_index = data[data.title==course_title]['index'].values[0]
    
    #Finding cosine similarity score with respect to an index number of given movie title which you like most
    similar_courses = list(enumerate(cosin_sim[course_index]))
    
    #Sorting cosine similarity scores in descending order
    sorted_similar_courses = sorted(similar_courses,key=lambda x:x[1], reverse=True)
    
    #Extracting index numbers from sorted cosine similarity scores
    courses_indices = [i[0] for i in sorted_similar_courses]
    
    #Extracting movies title with respect to extracted index numbers from sorted cosine similarity scores
    courses_name = data['title'].iloc[courses_indices]
    
    #Creating dataframe of recommended movies name and resettig the index number
    recommended_courses = pd.DataFrame()
    recommended_courses['Recommended Courses'] = courses_name
    recommended_courses = recommended_courses.reset_index(drop = True)
    
    #Showing the result
    return recommended_courses.iloc[0:10]

In [50]:
get_recommender('Linear Algebra')

,Recommended Courses
0,Differential Equations
1,Differential Equations
2,Differential Equations
3,Dynamical Systems
4,Dynamics of Rigid Bodies
5,Differential Equations and Modeling
6,Scientific Computing
7,Scientific Computing
8,Numerical Analysis
9,Computational Partial Differential Equations
